In [113]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs
from fileUtils import getDirBasics
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir
from pandasUtils import getRowDataByColValue

from difflib import SequenceMatcher
import operator
from glob import glob
from os.path import join

from masterdb import getArtistAlbumsDB, discConv


## Discogs
from discogsBase import discogs


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-02-07 13:29:25.253178


## Create Discogs Objects

In [2]:
discDC = discogs('discogs')
discAM = discogs("allmusic")

## Artist Data (AllMusic)

In [3]:
print("Getting Master Artist DB File (AllMusic)")
discdfAM = discAM.getMasterSlimArtistDiscogsDB()
artistsAM = [x for x in list(discdfAM["Artist"]) if x is not None]
print(len(artistsAM))

artistIDToNameAM = discdfAM["DiscArtist"].to_dict()
artistNameToIDAM = {}
print("Found {0} ID -> Name entries".format(len(artistIDToNameAM)))
for artistID,artistName in artistIDToNameAM.items():
    if artistName is None:
        continue
    if artistNameToIDAM.get(artistName) is None:
        artistNameToIDAM[artistName] = []
    artistNameToIDAM[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToIDAM)))

artistAlbumsDBAM = getArtistAlbumsDB(discAM, force=False)
print("Found {0} Artist Albums".format(artistAlbumsDBAM.shape[0]))

Getting Master Artist DB File (AllMusic)
36432
Found 36432 ID -> Name entries
Found 34373 Name -> ID entries
Current Time is Thu Feb 06, 2020 17:53:40 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Thu Feb 06, 2020 17:53:42 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 1 seconds.
Found 36432 Artist Albums


## Artist Data (Discogs)

In [150]:
print("Getting Master Artist DB File (Discogs)")
discdfDC = discDC.getMasterSlimArtistDiscogsDB()
artistsDC = [x for x in list(discdfDC["Artist"]) if x is not None]
print(len(artistsDC))

artistIDToNameDC = discdfDC["DiscArtist"].to_dict()
artistNameToIDDC = {}
print("Found {0} ID -> Name entries".format(len(artistIDToNameDC)))
for artistID,artistName in artistIDToNameDC.items():
    if artistName is None:
        continue
    if artistNameToIDDC.get(artistName) is None:
        artistNameToIDDC[artistName] = []
    artistNameToIDDC[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToIDDC)))

artistAlbumsDBDC = getArtistAlbumsDB(discDC, force=False)
print("Found {0} Artist Albums".format(artistAlbumsDBDC.shape[0]))

Getting Master Artist DB File (Discogs)
712587
Found 712629 ID -> Name entries
Found 654970 Name -> ID entries
Current Time is Fri Feb 07, 2020 15:25:18 for 
=================================== Creating Artist Albums DB ===================================
Using previously created Artist Albums DB
Current Time is Fri Feb 07, 2020 15:26:03 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [Done with 
=================================== Creating Artist Albums DB ===================================] took 44 seconds.
Found 712629 Artist Albums


## Multi Artist Objects

In [151]:
mulArtsAM  = multiArtist(cutoff=0.9, discdata=artistNameToIDAM, exact=False)
mulArtsDC  = multiArtist(cutoff=0.9, discdata=artistNameToIDDC, exact=False)

## Music Discogs Maps

In [6]:
## Do not execute this stuff

In [ ]:
try:
    myMusicNameIDMap = getFile("musicDiscogsMap.p")
    print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))
except:
    myMusicNameIDMap = {}
    print("Could not load music <-> discogs map")
       
myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']

In [ ]:
myMusicMap = {}
for myArtistName, discogsData in myMusicNameIDMap.items():
    if myMusicMap.get(myArtistName) is None:
        myMusicMap[myArtistName] = {"Discogs": None, "AllMusic": None}
    myMusicMap[myArtistName]["Discogs"] = {"ID": discogsData[0], "Name": discogsData[1]}
print("There are {0} my music data".format(len(myMusicMap)))

In [ ]:
artistAMIDtoDCIDMap = getFile(ifile="/Users/tgadfort/Documents/code/discogs/artistAMIDtoDCIDMap.p")
artistDCIDtoAMIDMap = {v["DC ID"]: k for k,v in artistAMIDtoDCIDMap.items()}
print("There are {0} AM Artist ID to DC entries".format(len(artistAMIDtoDCIDMap)))
print("There are {0} DC Artist ID to AM entries".format(len(artistDCIDtoAMIDMap)))

In [ ]:
for myArtistName, discogsData in myMusicNameIDMap.items():
    if myMusicMap.get(myArtistName) is None:
        myMusicMap[myArtistName] = {"Discogs": None, "AllMusic": None}
    artistAMData = artistDCIDtoAMIDMap.get(discogsData[0])
    if artistAMData is not None:
        myMusicMap[myArtistName]["AllMusic"] = {"ID": artistAMData, "Name": None}
print("There are {0} my music data".format(len(myMusicMap)))

In [ ]:
saveFile(idata=myMusicMap, ifile="myMusicMap.p")

# Fix Discogs Mapping

In [ ]:
def fixDiscogsMapping(myMusicNameIDMap):
    musicNameIDMap = {}
    for artistName,v in myMusicNameIDMap.items():
        if isinstance(v, list):
            musicNameIDMap[artistName] = v
            #print(artistName,v)
            continue

        mdata = getMusicData("Name", v)
        if isinstance(mdata, DataFrame):
            if mdata.shape[0] == 1:
                #print(artistName,[mdata.index[0], mdata["Name"].values[0]])
                musicNameIDMap[artistName] = [mdata.index[0], mdata["Name"].values[0]]
            else:
                raise ValueError("No idea about {0}".format(mdata))
        else:
            print(artistName,'\t',v)
            
    return musicNameIDMap
        
#myMusicNameIDMap = musicNameIDMap
#from ioUtils import saveFile
#saveFile(ifile="musicDiscogsMap.p", idata=musicNameIDMap, debug=True)

In [ ]:
myMusicNameIDMap = fixDiscogsMapping(myMusicNameIDMap)

In [ ]:
if myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']:
    saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
    saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

In [ ]:
manualMatches = {}

if len(manualMatches) > 0:
    myMusicNameIDMap.update(manualMatches) 

print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

# Find Music and Match Artists

In [66]:
myMusicMap = getFile(ifile="myMusicMap.p")
print("Found {0} entries".format(len(myMusicMap)))
cntrs = {"Discogs": 0, "AllMusic": 0}
for myArtistName, myArtistData in myMusicMap.items():
    if myArtistData["Discogs"] is not None:
        cntrs["Discogs"] += 1
    if myArtistData["AllMusic"] is not None:
        cntrs["AllMusic"] += 1
print(cntrs)

#Found 4692 entries
#{'Discogs': 4692, 'AllMusic': 4005}

Found 4692 entries
{'Discogs': 4692, 'AllMusic': 4233}


## Loop over My Music

In [159]:
from pandas import DataFrame

musicNameIDMap = {}
multiMap       = {}


def getArtistAlbums(discdf, idx):
    if not isinstance(discdf, DataFrame):
        raise ValueError("Not a DataFrame")
    
    try:
        artistAlbumsData = discdf[discdf.index == idx]
        artistAlbums     = artistAlbumsData["Albums"].to_dict().get(idx)
    except:
        return {}
        
    return artistAlbums


def getArtistIDX(artistMapData, name, discdf):
    if not isinstance(artistMapData, dict):
        raise ValueError("No Artist Map Data")
    if not isinstance(discdf, DataFrame):
        raise ValueError("Not a DataFrame")        
    if not isinstance(artistMapData, dict):
        raise ValueError("artistMapData is not a DB!")
    
    try:
        idx = artistMapData.get(name).get('ID')
    except:
        return None
    
    ## Check
    if idx not in list(discdf.index):
        raise ValueError("ID {0} for {1} is not in the Index of the main DataFrame!".format(idx, name))
    
    return idx

def getArtistIDDBCounts(dbIDData):
    if isinstance(discogsIDData, DataFrame):
        return dbIDData.shape[0]
    return 0

def printArtistIDs(artistName, discogsArtistIDX, allmusicArtistIDX):
    print('\t{0: <40}{1: <15}{2: <15}'.format(artistName,str(discogsArtistIDX),str(allmusicArtistIDX)))

def printArtistIDDBResults(artistName, discogsIDData, allmusicIDData):
    print("\t{0: <40}{1: <15}{2: <15}".format("", 
                                              getArtistIDDBCounts(discogsIDData),
                                              getArtistIDDBCounts(allmusicIDData)))
    
def getMyMusicAlbums(dirval, returnNames=False):    
    discogMediaNames   = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
    allmusicMediaNames = ['Album']
    myMediaNames       = ['Random', 'Todo', 'Match', 'Title', 'Singles']
    
    myMusicAlbums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in discogMediaNames+allmusicMediaNames+myMediaNames]
    if returnNames is True:
        myMusicAlbums = [getDirBasics(x)[-1] for x in myMusicAlbums]
    return myMusicAlbums


def getMyMatchedMusicAlbums(dirval):  
    matchval = join(dirval, "Match", "*")
    matchedAlbums = []
    for dname in glob(matchval):
        matchedAlbums += [getDirBasics(x)[-1].split(" :: ")[0] for x in findDirs(dname)]
    return matchedAlbums


def getMyDiscogsMusicAlbums(dirval):
    discogsAlbums      = []
    discogMediaNames   = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
    for discogName in discogMediaNames:
        matchval = join(dirval, discogName, "*")
        for dname in glob(matchval):
            discogsAlbums += [getDirBasics(x)[-1] for x in findDirs(dname)]
    return discogsAlbums



def getFlattenedArtistAlbums(vals):
    if vals is None or not isinstance(vals, dict):
        return []
    albums = []
    for k,v in vals.items():
        for k2, v2 in v.items():
            albums.append(v2)
    return list(set(albums))
            
    
def getBestAlbumsMatch(albums1, albums2, cutoff=0.7, debug=False):
    if debug:
        print("Best Albums Match")
    score = 0.0
    for albumA in albums1:
        for albumB in albums2:
            s = SequenceMatcher(None, albumA, albumB)
            ratio = round(s.ratio(), 2)
            if debug:
                print("\t\t\t[{0}] + [{1}] = [{2}]".format(albumA, albumB, ratio))
            if ratio > cutoff:
                score += ratio
    return score


def findPossibleArtistIDs(artistName, artistNameToID, artists, num=2, cutoff=0.7):
    possibleIDs    = artistNameToID.get(artistName)
    if possibleIDs is None:
        possibleIDs = []
    print(artistName,'\t',len(possibleIDs),'\t',len(artists))
    newArtistNames = findNearest(artistName, artists, num, cutoff)
    for newArtist in newArtistNames:
        possibleIDs += artistNameToID[newArtist]
    
    possibleIDs  = list(set(possibleIDs))

    return possibleIDs

## Find Known Music Artists

In [136]:
start, cmt = clock("Matching All Music Artists")

fullyUnknownArtistNames     = {}
partiallyUnknownArtistNames = {}
multiKnownArtistNames       = {}

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        
        ## Get My Music Artist Name
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        ## Get Database IDs
        myMusicData          = myMusicMap.get(artistName)
        if myMusicData is None:
            fullyUnknownArtistNames[artistName] = dirval
            print("\tUnknown (All)     --> {0}".format(artistName))
            continue
        discogsArtistIDX     = getArtistIDX(myMusicData, "Discogs", discdfDC)
        allmusicArtistIDX    = getArtistIDX(myMusicData, "AllMusic", discdfAM)
        
        
        if all([discogsArtistIDX, allmusicArtistIDX]):
            ## Previously Known By All DBs
            print("\tKnown (All)       --> {0}".format(artistName))
            continue
        
        if not any([discogsArtistIDX, allmusicArtistIDX]):
            ## Previously UnKnown By All DBs
            fullyUnknownArtistNames[artistName] = dirval
            print("\tUnknown (All)     --> {0}".format(artistName))
            continue
            
        partiallyUnknownArtistNames[artistName] = {}
        if discogsArtistIDX is None:
            partiallyUnknownArtistNames[artistName]["Discogs"] = dirval
        if allmusicArtistIDX is None:
            partiallyUnknownArtistNames[artistName]["AllMusic"] = dirval
        print("\tUnknown (Partial) --> {0}".format(artistName))        
        
elapsed(start, cmt)

Current Time is Fri Feb 07, 2020 15:07:25 for Matching All Music Artists
/Users/tgadfort/matched/A
	Known (All)       --> A Flock Of Seagulls
	Known (All)       --> A Global Threat
	Known (All)       --> A Great Big World
	Known (All)       --> A Tribe Called Quest
	Known (All)       --> A*Teens
	Unknown (Partial) --> A-Mafia
	Known (All)       --> A-Set
	Known (All)       --> Aaron Neville
	Known (All)       --> ABBA
	Known (All)       --> ABC
	Known (All)       --> Abdullah
	Known (All)       --> Abigor
	Known (All)       --> Abominator
	Known (All)       --> Above The Law
	Known (All)       --> Abscess
	Known (All)       --> Absolution
	Known (All)       --> Absurdus
	Known (All)       --> Abwärts
	Known (All)       --> AC-DC
	Known (All)       --> Accept
	Known (All)       --> Ace Of Base
	Known (All)       --> Acker Bilk
	Known (All)       --> Action League
	Known (All)       --> Adam And The Ants
	Known (All)       --> Adam Ant
	Known (All)       --> Adam Lambert
	Known (All)    

	Known (All)       --> Barcode Brothers
	Known (All)       --> Barenaked Ladies
	Known (All)       --> Barracudas
	Known (All)       --> Barry Berrier
	Unknown (Partial) --> Barry Kleinbort
	Known (All)       --> Barry Manilow
	Known (All)       --> Barry White
	Known (All)       --> Basement Jaxx
	Known (All)       --> Basic Element
	Unknown (Partial) --> Bass
	Known (All)       --> Bassholes
	Known (All)       --> Battlelust
	Known (All)       --> BBMak
	Known (All)       --> Beachbuggy
	Unknown (Partial) --> Beam & Yanou
	Known (All)       --> Beanfield
	Known (All)       --> Beanie Sigel
	Known (All)       --> Beastie Boys
	Unknown (Partial) --> Beat Guru
	Unknown (Partial) --> Beatrice Arthur
	Known (All)       --> Beaver Nelson
	Unknown (Partial) --> Beavis & Butthead
	Known (All)       --> Bebe Rebozo
	Known (All)       --> Beck
	Known (All)       --> Beck, Bogert & Appice
	Known (All)       --> Becky G
	Known (All)       --> Becky Hobbs
	Unknown (Partial) --> Bedřich Smetana
	K

	Known (All)       --> Boyzone
	Known (All)       --> Brad Gillis
	Known (All)       --> Braid
	Known (All)       --> Brainstorm
	Known (All)       --> Brainticket
	Known (All)       --> Brand New
	Known (All)       --> Brand X
	Unknown (Partial) --> Brandon Cruz
	Known (All)       --> Brandy Clark
	Known (All)       --> Branford Marsalis
	Known (All)       --> Brantley Gilbert
	Known (All)       --> Bratmobile
	Known (All)       --> Brazil
	Unknown (Partial) --> Brazil 2001
	Known (All)       --> Breach
	Known (All)       --> Brenda Kahn
	Known (All)       --> Brenda Lee
	Known (All)       --> Brendan Benson
	Known (All)       --> Bria Valente
	Known (All)       --> Brian Hyland
	Known (All)       --> Brian Leach
	Known (All)       --> Brian Setzer
	Known (All)       --> Brian Setzer Orchestra
	Known (All)       --> Bride Of No No
	Known (All)       --> Bright
	Known (All)       --> Britney Spears
	Known (All)       --> Broken Hearts Are Blue
	Known (All)       --> Broken Hope
	Known 

	Known (All)       --> Clan Of Xymox
	Known (All)       --> Clarence Carter
	Known (All)       --> Clark Gesner
	Known (All)       --> Claw Hammer
	Known (All)       --> Clay Aiken
	Known (All)       --> Clay Walker
	Known (All)       --> Clean Bandit
	Known (All)       --> Cleanser
	Known (All)       --> Cleen
	Known (All)       --> Clem Snide
	Known (All)       --> Cletus
	Known (All)       --> Cleveland Bound Death Sentence
	Known (All)       --> Cliff Eberhardt
	Known (All)       --> Cliff Richard
	Known (All)       --> Clifters
	Known (All)       --> Clint Black
	Known (All)       --> Clinton Sparks
	Known (All)       --> Clipse
	Known (All)       --> Clocked In
	Known (All)       --> Clowns For Progress
	Known (All)       --> CLS
	Known (All)       --> Club 8
	Known (All)       --> Clutch
	Known (All)       --> Co-Ed
	Known (All)       --> Coalesce
	Known (All)       --> Cobraman
	Known (All)       --> Cock Sparrer
	Known (All)       --> Cockney Rejects
	Known (All)       --> Cof

	Known (All)       --> Daylight Dies
	Known (All)       --> DC Talk
	Known (All)       --> De De
	Known (All)       --> De La Soul
	Known (All)       --> De Nattergale
	Unknown (Partial) --> De Profundis
	Known (All)       --> Dead 50's
	Known (All)       --> Dead And Gone
	Known (All)       --> Dead Horse
	Known (All)       --> Dead Kennedys
	Known (All)       --> Dead Meadow
	Known (All)       --> Dead Moon
	Known (All)       --> Dead Or Alive
	Known (All)       --> Dead Ringers
	Known (All)       --> Dead Sheriff
	Known (All)       --> Dead World
	Known (All)       --> Deadguy
	Known (All)       --> Deadspot
	Known (All)       --> Deal's Gone Bad
	Known (All)       --> Dean Martin
	Known (All)       --> Deana Carter
	Known (All)       --> Death By Chocolate
	Known (All)       --> Death By Stereo
	Known (All)       --> Death Cab For Cutie
	Known (All)       --> Death Ride 69
	Known (All)       --> Death Threat
	Known (All)       --> Deathwitch
	Known (All)       --> Debbie Deb
	Known

...

	Known (All)       --> Them Wranch
	Known (All)       --> Therion
	Unknown (Partial) --> Thermocline
	Unknown (Partial) --> Therr Maitz
	Known (All)       --> These 5 Down
	Known (All)       --> Theta Wave State
	Known (All)       --> They Come In Threes
	Known (All)       --> They Might Be Giants
	Known (All)       --> Thin Lizzy
	Known (All)       --> Thin White Rope
	Known (All)       --> Thinking Man
	Known (All)       --> Third Age
	Known (All)       --> Third Ear Band
	Known (All)       --> Third Eye Blind
	Known (All)       --> Thirdmoon
	Known (All)       --> Thirstin Howl III
	Known (All)       --> This Day Forward
	Unknown (Partial) --> This World Rejected
	Known (All)       --> Thomas Bangalter
	Known (All)       --> Thomas Helmig
	Known (All)       --> Thomas Newman
	Known (All)       --> Thomas Rhett
	Known (All)       --> Thomas Rusiak
	Known (All)       --> Thompson Twins
	Known (All)       --> Thorr's Hammer
	Known (All)       --> Thou Shalt Suffer
	Known (All)       --

	Known (All)       --> The Busy Signals
	Known (All)       --> The Butchies
	Known (All)       --> The Byrds
	Known (All)       --> The Call
	Known (All)       --> The Cambridge Singers
	Unknown (Partial) --> The Campus Tramps
	Known (All)       --> The Capitols
	Known (All)       --> The Cardigans
	Known (All)       --> The Cars
	Known (All)       --> The Carter Family
	Known (All)       --> The Casket Lottery
	Known (All)       --> The Casualties
	Known (All)       --> The Catheters
	Known (All)       --> The Causey Way
	Known (All)       --> The Cells
	Known (All)       --> The Chainsmokers
	Known (All)       --> The Challengers
	Known (All)       --> The Chameleons
	Known (All)       --> The Charlie Daniels Band
	Known (All)       --> The Charlottes
	Known (All)       --> The Chase Theory
	Known (All)       --> The Chemical Brothers
	Known (All)       --> The Chesterf!elds
	Known (All)       --> The Chicken Hawks
	Known (All)       --> The Chieftains
	Known (All)       --> The Chil

	Known (All)       --> The King's Singers
	Known (All)       --> The Kinks
	Known (All)       --> The Kinleys
	Known (All)       --> The Kirby Grips
	Known (All)       --> The Kiss Offs
	Known (All)       --> The Kowalskis
	Known (All)       --> The Kreeg
	Known (All)       --> The Krontjong Devils
	Known (All)       --> The Lapse
	Known (All)       --> The Last
	Known (All)       --> The Laurie Berkner Band
	Known (All)       --> The Lazy Cowgirls
	Known (All)       --> The League Unlimited Orchestra
	Known (All)       --> The Legendary Invisible Men
	Known (All)       --> The Legendary Stardust Cowboy
	Known (All)       --> The Lemon Drops
	Known (All)       --> The Lemonheads
	Known (All)       --> The Levellers
	Known (All)       --> The Liberty Bell
	Known (All)       --> The Light Crust Doughboys
	Known (All)       --> The Lilac Time
	Known (All)       --> The Limeliters
	Known (All)       --> The Lincoln Center Jazz Orchestra
	Known (All)       --> The Lisa Marr Experiment
	Know

	Known (All)       --> The Stool Pigeons
	Known (All)       --> The Streetwalkin' Cheetahs
	Known (All)       --> The String Cheese Incident
	Unknown (Partial) --> The String Quartet
	Known (All)       --> The Stuck Ups
	Known (All)       --> The Suicide Kings
	Known (All)       --> The Sundays
	Known (All)       --> The Supremes
	Known (All)       --> The Surf Trio
	Known (All)       --> The Swaggerts
	Unknown (Partial) --> The Sweet
	Known (All)       --> The Swell Season
	Known (All)       --> The Swingin' Neckbreakers
	Known (All)       --> The Technical Jed
	Unknown (Partial) --> The Tell-Tale Hearts
	Known (All)       --> The Temptations
	Unknown (Partial) --> The Tennessee Tearjerkers
	Known (All)       --> The Three 4 Tens
	Known (All)       --> The Three O'Clock
	Known (All)       --> The Thrown Ups
	Known (All)       --> The Tiger Lillies
	Known (All)       --> The Tiki Tones
	Known (All)       --> The Timeout Drawer
	Known (All)       --> The Titanics
	Known (All)       --> 

	Known (All)       --> Wilco
	Unknown (Partial) --> Wild & Blue
	Known (All)       --> Wild Colonials
	Known (All)       --> Wilde Flowers
	Known (All)       --> Wildfire
	Unknown (Partial) --> Will I Am
	Known (All)       --> Will Johnson
	Known (All)       --> Will Smith
	Known (All)       --> William Orbit
	Known (All)       --> Willie Dixon
	Known (All)       --> Willie Nelson
	Known (All)       --> Willy Porter
	Known (All)       --> Wilson Phillips
	Known (All)       --> Windfall
	Known (All)       --> Windigo
	Known (All)       --> Winds Of Sirius
	Known (All)       --> Wingtip Sloat
	Known (All)       --> Wino
	Unknown (Partial) --> Winx
	Known (All)       --> Wisdom Of Harry
	Known (All)       --> Wishbone Ash
	Known (All)       --> Witch Hazel Sound
	Known (All)       --> Witchery
	Known (All)       --> Witchfynde
	Known (All)       --> Witchkiller
	Known (All)       --> Without Face
	Known (All)       --> Wiz Khalifa
	Known (All)       --> Wolf Eyes
	Known (All)       --> Wo

## Analyze Fully Unknown Music Artists

In [156]:
num = 2
cutoff = 0.6

discogMediaNames   = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
allmusicMediaNames = ['Album']
myMediaNames       = ['Random', 'Todo', 'Match', 'Title', 'Singles']

additions = {"Discogs": {}, "AllMusic": {}}

print("{0: <40}{1}".format("Artist", "# of Albums"))
for artistName, dirval in fullyUnknownArtistNames.items():
    myMusicAlbums = getMyMusicAlbums(dirval, returnNames=True) + getMyMatchedMusicAlbums(dirval) + getMyDiscogsMusicAlbums(dirval)
    if len(myMusicAlbums) == 0:
        continue
    print("{0: <40}{1}".format(artistName,len(myMusicAlbums)))

    for key in ["Discogs", "AllMusic"]:
        if key == "Discogs":
            possibleIDs    = findPossibleArtistIDs(artistName, artistNameToIDDC, artistsDC, num, cutoff)            
        elif key == "AllMusic":
            possibleIDs    = findPossibleArtistIDs(artistName, artistNameToIDAM, artistsAM, num, cutoff)            
        else:
            raise ValueError("DB Key [{0}] is not known!".format(key))


        print("     Possible IDs ===>",len(possibleIDs))
        for possibleID in possibleIDs:
            print("\t{0: <15}".format(possibleID), end="")
            if key == "Discogs":
                artistAlbums = getRowData(artistAlbumsDBDC, rownames=possibleID)['Albums']
            elif key == "AllMusic":
                artistAlbums = getRowData(artistAlbumsDBAM, rownames=possibleID)['Albums']            
            else:
                raise ValueError("DB Key [{0}] is not known!".format(key))

            artistAlbums = getFlattenedArtistAlbums(artistAlbums)          
            print("{1: <10}".format(allmusicID,len(artistAlbums)), end="")

            ## Find overlapping albums
            retval = getBestAlbumsMatch(artistAlbums, myMusicAlbums, cutoff=cutoff, debug=False)                
            print(round(retval,2))
            if retval > cutoff:
                if additions[key].get(artistName) is None:
                    additions[key][artistName] = {"Score": retval, "Key": key, "Value": {'ID': possibleID, 'Name': None}}
                elif retval > additions[key][artistName]["Score"]:
                    additions[key][artistName] = {"Score": retval, "Key": key, "Value": {'ID': possibleID, 'Name': None}}

                print("\t{0: <15} is a match!".format(possibleID))

print("")
print("Found {0} new matches".format(len(additions)))

Artist                                  # of Albums
Starship                                1
Starship 	 24 	 712587
     Possible IDs ===> 6
	245159         39        2.13
	245159          is a match!
	91266          2         0.0
	1786570        1         0.0
	5423159        1         0.0
	649435         1         0.0
	4407999        1         0.0
Starship 	 3 	 36432
     Possible IDs ===> 2
	0000018828     36        0.0
	0000748168     29        1.73
	0000748168      is a match!

Found 2 new matches


In [158]:
start, cmt = clock("Adding New Matches to myMusicMap")
for dbKey, dbKeyData in additions.items():
    for artistName, artistValues in dbKeyData.items():
        if myMusicMap.get(artistName) is None:
            myMusicMap[artistName] = {"Discogs": None, "AllMusic": None}
        myMusicMap[artistName][artistValues['Key']] = artistValues['Value']

cntrs = {"Discogs": 0, "AllMusic": 0}
for myArtistName, myArtistData in myMusicMap.items():
    if myArtistData["Discogs"] is not None:
        cntrs["Discogs"] += 1
    if myArtistData["AllMusic"] is not None:
        cntrs["AllMusic"] += 1
print(cntrs)

saveFile(idata=myMusicMap, ifile="myMusicMap.p", debug=True)
elapsed(start, cmt)

Current Time is Fri Feb 07, 2020 15:34:40 for Adding New Matches to myMusicMap
{'Discogs': 4693, 'AllMusic': 4303}
Saving data to myMusicMap.p
  --> This file is 196.4kB.
Saved data to myMusicMap.p
  --> This file is 196.4kB.
Current Time is Fri Feb 07, 2020 15:34:40 for Done with Adding New Matches to myMusicMap
Process [Done with Adding New Matches to myMusicMap] took 376.0 millseconds.


## Analyze Partially Unknown Music Artists

In [88]:
start, cmt = clock("Finding Possible New Matches")

num = 2
cutoff = 0.6


discogMediaNames   = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
allmusicMediaNames = ['Album']
myMediaNames       = ['Random', 'Todo', 'Match', 'Title', 'Singles']

additions = {}

from searchUtils import findNearest
print("{0: <40}{1}".format("Artist", "# of Albums"))
for artistName, unknownVals in partiallyUnknownArtistNames.items():
    
    key = "AllMusic"
    if unknownVals.get(key) is not None:
        dirval = unknownVals[key]
        
        myMusicAlbums = getMyMusicAlbums(dirval, returnNames=True) + getMyMatchedMusicAlbums(dirval) + getMyDiscogsMusicAlbums(dirval)
        if len(myMusicAlbums) == 0:
            continue
        print("{0: <40}{1}".format(artistName,len(myMusicAlbums)))
        
        if key == "Discogs":
            possibleIDs    = findPossibleArtistIDs(artistName, artistNameToIDDC, artistsDC, num, cutoff)            
        elif key == "AllMusic":
            possibleIDs    = findPossibleArtistIDs(artistName, artistNameToIDAM, artistsAM, num, cutoff)            
        else:
            raise ValueError("DB Key [{0}] is not known!".format(key))

            
        print("     Possible IDs ===>",len(possibleIDs))
        for possibleID in possibleIDs:
            print("\t{0: <15}".format(possibleID), end="")
            if key == "Discogs":
                artistAlbums = getRowData(artistAlbumsDBDC, rownames=possibleID)['Albums']
            elif key == "AllMusic":
                artistAlbums = getRowData(artistAlbumsDBAM, rownames=possibleID)['Albums']            
            else:
                raise ValueError("DB Key [{0}] is not known!".format(key))
            
            artistAlbums = getFlattenedArtistAlbums(artistAlbums)          
            print("{1: <10}".format(allmusicID,len(artistAlbums)), end="")
            
            ## Find overlapping albums
            retval = getBestAlbumsMatch(artistAlbums, myMusicAlbums, cutoff=cutoff, debug=False)                
            print(round(retval,2))
            if retval > cutoff:
                if additions.get(artistName) is None:
                    additions[artistName] = {"Score": retval, "Key": key, "Value": {'ID': possibleID, 'Name': None}}
                elif retval > additions[artistName]["Score"]:
                    additions[artistName] = {"Score": retval, "Key": key, "Value": {'ID': possibleID, 'Name': None}}
                    
                print("\t{0: <15} is a match!".format(possibleID))

print("")
print("Found {0} new matches".format(len(additions)))
elapsed(start, cmt)

Current Time is Fri Feb 07, 2020 11:19:07 for Finding Possible New Matches
Artist                                  # of Albums
A-Mafia                                 2
     Possible IDs ===> 2
	0001572547     1         0.0
	0000232101     27        0.0
Admiral D                               1
     Possible IDs ===> 2
	0000189497     6         0.0
	0002483942     7         0.0
Adryan Russ                             1
     Possible IDs ===> 2
	0000258514     18        0.0
	0000500904     2         0.0
Afrodite                                1
     Possible IDs ===> 2
	0001855410     1         0.0
	0002666663     2         0.0
Agent 99                                1
     Possible IDs ===> 2
	0000603742     3         0.0
	0000929073     1         0.0
Alley Boy                               1
     Possible IDs ===> 2
	0001584770     5         0.0
	0000847982     1         0.0
Apothys                                 1
     Possible IDs ===> 3
	0003862318     1         0.0
	0000546859   

     Possible IDs ===> 2
	0000145977     14        0.0
	0000424707     2         0.0
Go Nuts                                 1
     Possible IDs ===> 2
	0003571348     1         0.0
	0000558490     9         0.0
Gramsespektrum                          1
     Possible IDs ===> 0
Great Minds                             2
     Possible IDs ===> 2
	0002962520     2         0.0
	0000074147     1         0.0
Grimace                                 1
     Possible IDs ===> 2
	0000202658     1         0.0
	0003384447     3         0.0
Gwar                                    7
     Possible IDs ===> 2
	0000042292     9         0.0
	0000044379     1         0.0
H3llb3nt                                2
     Possible IDs ===> 0
HALO                                    1
     Possible IDs ===> 1
	0000660855     5         0.0
Harold Prince                           1
     Possible IDs ===> 2
	0000293890     2         0.0
	0000952516     7         0.0
Hildegard Von Bingen                    1
     Po

PUD                                     1
     Possible IDs ===> 1
	0000792868     3         0.0
Puke-a-Rama                             1
     Possible IDs ===> 0
Pure 13                                 1
     Possible IDs ===> 2
	0002682812     9         0.0
	0000857985     8         0.0
Rebel Gang                              1
     Possible IDs ===> 2
	0003221967     1         0.0
	0003887780     1         0.0
Ricky Skaggs & Friends                  1
     Possible IDs ===> 2
	0000981382     2         0.0
	0003486521     1         0.0
Rob Luna                                1
     Possible IDs ===> 2
	0000993435     1         0.0
	0001533723     1         0.0
Robert Solomon                          1
     Possible IDs ===> 2
	0000285693     36        0.0
	0001540902     1         0.0
Rodney Allen                            1
     Possible IDs ===> 2
	0000742990     48        0.0
	0000833923     1         0.0
Ron Paul                                1
     Possible IDs ===> 2
	000371

In [84]:
start, cmt = clock("Adding New Matches to myMusicMap")
for artistName, artistValues in additions.items():
    if myMusicMap.get(artistName) is None:
        myMusicMap[artistName] = {"Discogs": None, "AllMusic": None}
    myMusicMap[artistName][artistValues['Key']] = artistValues['Value']

cntrs = {"Discogs": 0, "AllMusic": 0}
for myArtistName, myArtistData in myMusicMap.items():
    if myArtistData["Discogs"] is not None:
        cntrs["Discogs"] += 1
    if myArtistData["AllMusic"] is not None:
        cntrs["AllMusic"] += 1
print(cntrs)

saveFile(idata=myMusicMap, ifile="myMusicMap.p", debug=True)
elapsed(start, cmt)

Current Time is Fri Feb 07, 2020 11:12:16 for Adding New Matches to myMusicMap
{'Discogs': 4692, 'AllMusic': 4302}
Saving data to myMusicMap.p
  --> This file is 196.4kB.
Saved data to myMusicMap.p
  --> This file is 196.4kB.
Current Time is Fri Feb 07, 2020 11:12:17 for Done with Adding New Matches to myMusicMap
Process [Done with Adding New Matches to myMusicMap] took 653.0 millseconds.


# Match Albums

In [184]:
def showArtist(artistName, discogsArtist):
    print("\t{0: <30} ---> {1}".format(artistName, discogsArtist))
    
def showAlbum(myAlbumName):
    print("\t\tMy Album: {0}".format(myAlbumName))
    
def showMatch(myAlbumName, albumData):
    print("\t\t   Match: {0}".format(albumData))
    
def matchMyAlbum(myAlbumName, dbAlbumsData, ratioCut):
    maxRatio  = None
    albumData = None
    for mediaType, mediaAlbums in dbAlbumsData.items():
        for albumID, dbAlbumName in mediaAlbums.items():
            convAlbumName = discConv(dbAlbumName)
            s = SequenceMatcher(None, myAlbumName, convAlbumName)
            ratio = round(s.ratio(), 3)
            if ratio > ratioCut:
                if maxRatio is None:
                    maxRatio = ratio
                else:
                    if ratio < maxRatio:
                        continue
                    maxRatio  = ratio
                    albumData = [albumID, dbAlbumName, mediaType]

                    albumData = [albumID, dbAlbumName, mediaType]
                    
    return albumData

In [189]:
ratioCut          = 0.7
dbKey = "AllMusic"
myMusicAlbumMatch = {dbKey: {}}

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        
        ## Get My Music Artist Name
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)


        ## Get Database IDs
        myMusicData          = myMusicMap.get(artistName)
        if myMusicData is None:
            raise ValueError("Found an unknown artist: [{0}]".format(artistName))
                
            
        ## My Albums
        myMusicAlbums = getMyMusicAlbums(dirval, returnNames=True)
        if len(myMusicAlbums) == 0:
            continue

        ## Discogs
        key = "Discogs"
        if dbKey == key:
            artistIDX = getArtistIDX(myMusicData, key, discdfDC)
            if artistIDX is not None:
                artistAlbumsDC = getRowData(artistAlbumsDBDC, rownames=artistIDX)['Albums']
                for myAlbumName in myMusicAlbums:
                    albumData = matchMyAlbum(myAlbumName, artistAlbumsDC, ratioCut)
                    if albumData is not None:
                        showArtist(artistName, artistIDX)
                        if myMusicAlbumMatch[key].get(artistName) is None:
                            myMusicAlbumMatch[key][artistName] = {}
                        showAlbum(myAlbumName)
                        myMusicAlbumMatch[key][artistName][myAlbumName] = albumData
                        showMatch(myAlbumName, albumData)
                    
                    
        ## AllMusic
        key = "AllMusic"
        if dbKey == key:
            artistIDX = getArtistIDX(myMusicData, key, discdfAM)
            if artistIDX is not None:
                artistAlbumsAM = getRowData(artistAlbumsDBAM, rownames=artistIDX)['Albums']
                for myAlbumName in myMusicAlbums:
                    albumData = matchMyAlbum(myAlbumName, artistAlbumsAM, ratioCut)
                    if albumData is not None:
                        showArtist(artistName, artistIDX)
                        if myMusicAlbumMatch[key].get(artistName) is None:
                            myMusicAlbumMatch[key][artistName] = {}
                        showAlbum(myAlbumName)
                        myMusicAlbumMatch[key][artistName][myAlbumName] = albumData
                        showMatch(myAlbumName, albumData)

/Users/tgadfort/matched/A
/Users/tgadfort/matched/B
		My Album: Fire on the floor
		   Match: ['0002978900', 'Fire on the Floor', 'Single/EP']
		My Album: Live At Paradiso
		   Match: ['0001004328', 'Live at Paradiso', 'Video']
		My Album: Live At The Royal Albert Hall
		   Match: ['0003210119', 'Live at the Royal Albert Hall', 'Comp']
		My Album: Greatest Hits Vol. I
		   Match: ['0000024800', 'Greatest Hits, Vol. 3', 'Comp']
/Users/tgadfort/matched/C
/Users/tgadfort/matched/D
/Users/tgadfort/matched/E
/Users/tgadfort/matched/F
/Users/tgadfort/matched/G
		My Album: Pain Killers
		   Match: ['0003302237', 'Pain Killers: Reloaded', 'Album']
/Users/tgadfort/matched/H
/Users/tgadfort/matched/I
/Users/tgadfort/matched/J
		My Album: Raw Soul
		   Match: ['0000490823', 'R&B Soul', 'Comp']
		My Album: In The Studio
		   Match: ['0000542984', 'In the Studio, Vol. 3', 'Comp']
/Users/tgadfort/matched/K
/Users/tgadfort/matched/L
/Users/tgadfort/matched/M
/Users/tgadfort/matched/N
/Users/tgadfort/

### Manually Check If Match is Correct

In [190]:
from ioUtils import saveFile
saveFile(ifile="myMusicAlbumMatch.yaml", idata=myMusicAlbumMatch, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumMatch)))

Saving data to myMusicAlbumMatch.yaml
Saved data to myMusicAlbumMatch.yaml
  --> This file is 848B.
Found 1 music <-> discogs albums maps


In [191]:
myMusicAlbumMatch = getFile(ifile="myMusicAlbumMatch.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumMatch)))

Loading data from myMusicAlbumMatch.yaml
  --> This file is 275B.
Loading myMusicAlbumMatch.yaml
Found 1 music <-> discogs albums maps


# Move To Matched

In [192]:
def getPrimeDirectory(artistName):
    start = artistName[0]

    import string
    if start in string.ascii_uppercase:
        if artistName.startswith("The "):
            return "The"
        return start
    if start in string.ascii_lowercase:
        return "Xtra"
    elif start in string.digits:
        return "Num"
    else:
        raise ValueError("Could not determine Prime Directory for Artist {0}".format(artistName))
    
    
baseDir = "/Users/tgadfort/matched"
for dbKey, dbValues in myMusicAlbumMatch.items():
    for artistName, artistAlbums in dbValues.items():
        print("==>",artistName)
        dirval = join(baseDir, getPrimeDirectory(artistName), artistName)
        matchedDir = setDir(dirval, "Match")
        mkDir(matchedDir)
        
        for myAlbumName,albumVal in artistAlbums.items():
            albumID, dbAlbumName, mediaType = albumVal
            
            srcDir = setDir(dirval, myAlbumName)
            if not isDir(srcDir):
                print("{0} does not exist".format(srcDir))
                continue
                
            mediaDir = setDir(matchedDir, discConv(mediaType))
            mkDir(mediaDir)
            
            if dbKey == "Discogs":
                dstName = " :: ".join([discConv(myAlbumName), "[{0}]".format(albumID)])
            elif dbKey == "AllMusic":
                dstName = " :: ".join([discConv(myAlbumName), "[{0}] AM".format(albumID)])           
            else:
                raise ValueError("DB Key [{0}] is not known!".format(key))

            dstDir = setDir(mediaDir, dstName)
            if isDir(dstDir):
                print("{0} already exists".format(dstDir))
                continue

            print("\tMoving {0}  --->  {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir, debug=True)

==> Beth Hart
	Moving /Users/tgadfort/matched/B/Beth Hart/Fire on the floor  --->  /Users/tgadfort/matched/B/Beth Hart/Match/Single-EP/Fire on the floor :: [0002978900] AM
Moving /Users/tgadfort/matched/B/Beth Hart/Fire on the floor to /Users/tgadfort/matched/B/Beth Hart/Match/Single-EP/Fire on the floor :: [0002978900] AM
	Moving /Users/tgadfort/matched/B/Beth Hart/Live At Paradiso  --->  /Users/tgadfort/matched/B/Beth Hart/Match/Video/Live At Paradiso :: [0001004328] AM
Moving /Users/tgadfort/matched/B/Beth Hart/Live At Paradiso to /Users/tgadfort/matched/B/Beth Hart/Match/Video/Live At Paradiso :: [0001004328] AM
	Moving /Users/tgadfort/matched/B/Beth Hart/Live At The Royal Albert Hall  --->  /Users/tgadfort/matched/B/Beth Hart/Match/Comp/Live At The Royal Albert Hall :: [0003210119] AM
Moving /Users/tgadfort/matched/B/Beth Hart/Live At The Royal Albert Hall to /Users/tgadfort/matched/B/Beth Hart/Match/Comp/Live At The Royal Albert Hall :: [0003210119] AM


## Analyze Number of Albums

In [183]:
print("{0: <40}{1: <10}{2: <10}{3: <10}{4: <40}{5: <40}".format("Artist", "Albums", "Discogs", "AllMusic", "No Albums", "Unmatched"))
for dirN in findDirs("/Users/tgadfort/matched"):
    for dirval in findDirs(dirN):
        
        ## Get My Music Artist Name
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        
        ## My Albums
        myMusicAlbums        = getMyMusicAlbums(dirval, returnNames=True)
        myMatchedMusicAlbums = getMyMatchedMusicAlbums(dirval)
        print("{0: <40}".format(artistName), end="")
        print("{0: <10}".format(str((len(myMusicAlbums),len(myMatchedMusicAlbums)))), end="")

        ## Get Database IDs
        myMusicData          = myMusicMap.get(artistName)
        if myMusicData is None:
            raise ValueError("Found an unknown artist: [{0}]".format(artistName))
            
            
        for key in ["Discogs", "AllMusic"]:
            artistAlbums = []
            if key == "Discogs":
                artistID     = getArtistIDX(myMusicData, key, discdfDC)
                if artistID is not None:
                    artistAlbums = getRowData(artistAlbumsDBDC, rownames=artistID)['Albums']
                    artistAlbums = getFlattenedArtistAlbums(artistAlbums)
                    print("{0: <10}".format(len(artistAlbums)), end="")
                else:
                    print("{0: <10}".format("---"), end="")
            elif key == "AllMusic":
                artistID     = getArtistIDX(myMusicData, key, discdfAM)
                if artistID is not None:
                    artistAlbums = getRowData(artistAlbumsDBAM, rownames=artistID)['Albums']
                    artistAlbums = getFlattenedArtistAlbums(artistAlbums)
                    print("{0: <10}".format(len(artistAlbums)), end="")
                else:
                    print("{0: <10}".format("---"), end="")
            else:
                raise ValueError("DB Key [{0}] is not known!".format(key))
            
        if len(myMusicAlbums)+len(myMatchedMusicAlbums) == 0:
            print("{0: <40}".format(artistName), end="")
        else:
            print("{0: <40}".format(""), end="")

        if len(myMusicAlbums) >= 2:
            print("{0: <40}".format(artistName), end="")
        else:
            print("{0: <40}".format(""), end="")

        print("")

Artist                                  Albums    Discogs   AllMusic  No Albums                               Unmatched                               
A Flock Of Seagulls                     (16, 5)   65        40                                                A Flock Of Seagulls                     
A Global Threat                         (0, 2)    11        7                                                                                         
A Great Big World                       (0, 0)    6         6         A Great Big World                                                               
A Tribe Called Quest                    (0, 7)    59        35                                                                                        
A*Teens                                 (0, 0)    30        27        A*Teens                                                                         
A-Mafia                                 (2, 0)    5         ---                               

Alannah Myles                           (1, 4)    26        15                                                                                        
Albert Ammons                           (0, 0)    65        27        Albert Ammons                                                                   
Albert Collins                          (1, 0)    56        60                                                                                        
Albert Hammond                          (0, 1)    112       32                                                                                        
Alberta Cross                           (1, 0)    14        9                                                                                         
Alberte                                 (0, 0)    1         ---       Alberte                                                                         
Alejandro Sanz                          (4, 0)    108       67                                

Angelo Moore                            (1, 0)    11        12                                                                                        
Angie Stone                             (1, 0)    57        32                                                                                        
Angizia                                 (0, 2)    10        5                                                                                         
Ani DiFranco                            (0, 2)    90        41                                                                                        
Animaniacs                              (1, 0)    9         8                                                                                         
Animotion                               (0, 0)    17        7         Animotion                                                                       
Anita Baker                             (0, 1)    48        34                                

Asleep At The Wheel                     (1, 0)    93        66                                                                                        
Aspera Ad Astra                         (1, 1)    4         2                                                                                         
Assemblage 23                           (0, 2)    20        15                                                                                        
Assert                                  (0, 1)    15        3                                                                                         
Assück                                  (1, 0)    10        2                                                                                         
Astral Projection                       (0, 0)    48        1         Astral Projection                                                               
Astroline                               (0, 0)    8         3         Astroline               

Badfinger                               (0, 1)    62        35                                                                                        
Bagman                                  (0, 1)    3         2                                                                                         
Baha Men                                (0, 0)    35        29        Baha Men                                                                        
Balaam And The Angel                    (0, 1)    21        9                                                                                         
Balearic Bill                           (0, 0)    1         1         Balearic Bill                                                                   
Baltimora                               (0, 0)    24        5         Baltimora                                                                       
Bambee                                  (0, 0)    7         1         Bambee                  

...

Under The Gun                           (0, 2)    3         3                                                                                         
Underdog                                (0, 1)    6         3                                                                                         
Underworld                              (0, 0)    185       96        Underworld                                                                      
Unearth                                 (0, 1)    19        14                                                                                        
Universitetskoret Lille MUKO            (0, 0)    12        ---       Universitetskoret Lille MUKO                                                    
Unpure                                  (0, 1)    11        2                                                                                         
Unruh                                   (0, 1)    6         1                                 

Vietnom                                 (0, 1)    7         4                                                                                         
Viewfinder                              (0, 1)    1         1                                                                                         
Viking Crown                            (0, 1)    4         2                                                                                         
Village People                          (0, 0)    130       49        Village People                                                                  
Vim Patior                              (0, 1)    1         1                                                                                         
Vince Gill                              (2, 1)    95        79                                                Vince Gill                              
Vince Guaraldi Trio                     (0, 1)    22        14                                

Weezer                                  (2, 4)    121       67                                                Weezer                                  
Weird Summer                            (2, 1)    6         6                                                 Weird Summer                            
Wendy & Lisa                            (0, 1)    36        10                                                                                        
Wesley Willis                           (1, 2)    34        26                                                                                        
Wesley Willis And The Dragnews          (0, 1)    15        26                                                                                        
Westlife                                (7, 38)   101       70                                                Westlife                                
Westside Connection                     (0, 0)    11        9         Westside Connection     

Wreck                                   (0, 1)    6         4                                                                                         
Wreckless Eric                          (0, 1)    40        23                                                                                        
Written in the Sand                     (1, 0)    1         1                                                                                         
Wu-Tang Clan                            (1, 1)    124       73                                                                                        
Wyclef Jean                             (1, 0)    120       40                                                                                        
Wynonna                                 (0, 2)    57        10                                                                                        
Wÿtchÿker                               (1, 0)    1         1                                 

Yume Bitsu                              (0, 2)    8         5                                                                                         
Yumi Arai                               (0, 0)    29        20        Yumi Arai                                                                       
Yung Joey                               (1, 0)    3         ---                                                                                       
Yuppicide                               (0, 3)    12        7                                                                                         
Yves Deruyter                           (0, 0)    45        27        Yves Deruyter                                                                   
Zac Brown Band                          (6, 1)    17        33                                                Zac Brown Band                          
Zakary Thaks                            (0, 1)    14        8                                 

In [125]:
## Accidental Moves

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        
        ## Get My Music Artist Name
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
                
        discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']        
        matchvals = glob(join(dirval, "Match", "*"))
        matchvals = [x for x in matchvals if getDirBasics(x)[-1] not in discogMediaNames]
        
        for src in matchvals:
            dst = src.replace("/Match/", "/")
            moveDir(src, dst, debug=True)

/Users/tgadfort/matched/A
Moving /Users/tgadfort/matched/A/Aqua/Match/Cartoon Heroes - The Best Of Aqua to /Users/tgadfort/matched/A/Aqua/Cartoon Heroes - The Best Of Aqua
/Users/tgadfort/matched/B
Moving /Users/tgadfort/matched/B/Beck/Match/Guero to /Users/tgadfort/matched/B/Beck/Guero
Moving /Users/tgadfort/matched/B/Beyoncé/Match/I Am... Sasha Fierce to /Users/tgadfort/matched/B/Beyoncé/I Am... Sasha Fierce
Moving /Users/tgadfort/matched/B/Björk/Match/Biophilia to /Users/tgadfort/matched/B/Björk/Biophilia
Moving /Users/tgadfort/matched/B/Björk/Match/Selmasongs to /Users/tgadfort/matched/B/Björk/Selmasongs
/Users/tgadfort/matched/C
Moving /Users/tgadfort/matched/C/Céline Dion/Match/A New Day... Live In Las Vegas to /Users/tgadfort/matched/C/Céline Dion/A New Day... Live In Las Vegas
Moving /Users/tgadfort/matched/C/Céline Dion/Match/C'est Pour Toi to /Users/tgadfort/matched/C/Céline Dion/C'est Pour Toi
Moving /Users/tgadfort/matched/C/Céline Dion/Match/For You to /Users/tgadfort/matc

In [ ]:
baseDir = "/Users/tgadfort/matched"
for dbKey, dbValues in myMusicAlbumMatch.items():
    for artistName, artistAlbums in dbValues.items():
        print("==>",artistName)
        start = artistName[0]
        dirval = join(baseDir, getPrimeDirectory(artistName), artistName)
        matchedDir = setDir(dirval, "Match")
        mkDir(matchedDir)
        
        for myAlbumName,albumVal in artistAlbums.items():
            srcDir = setDir(dirval, myAlbumName)
            if not isDir(srcDir):
                print("{0} does not exist".format(srcDir))
                continue
            dstDir = setDir(matchedDir, discConv(myAlbumName))
            if isDir(dstDir):
                print("{0} already exists".format(dstDir))
                continue

            print("\tMoving {0}  --->  {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir, debug=True)

In [ ]:

            
    ## Loop over Artist AM IDs
    for iAMID,artistAMID in enumerate(artistAMIDs):
        if artistAMIDtoDCIDMap.get(artistAMID) is not None:
            continue
        
        showNames(artistAMName, artistDCNames)
        artistAMAlbums = getArtistAlbums(artistAMToAlbumsData.get(artistAMID))
        if artistAMAlbums is None:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, "NONE"))
        else:
            print("\tAM ID={0: <15}{1: <5}/{2: <8}{3: <5}{4}".format(artistAMID, iKey, len(artistAMKeys), iAMID, len(artistAMAlbums)))

            
        artistAMtoDCScore = None
        
            
        ## Loop over Artist DC Names
        for artistDCName in artistDCNames:
            artistDCIDs = artistDCNameToID[artistDCName]
            
            ## Loop over Artist DC IDs
            for artistDCID in artistDCIDs:
                artistDCAlbums = getArtistAlbums(artistDCToAlbumsData.get(artistDCID))
                if False:
                    if artistDCAlbums is None:
                        print("\t\tDC ID={0: <15}\t      {1}".format(artistDCID, "NONE"))
                    else:
                        print("\t\tDC ID={0: <15}\t Size={1}".format(artistDCID, len(artistDCAlbums)))
                
                retval = getBestAlbumsMatch(artistAMAlbums, artistDCAlbums, debug=False)
                if retval > 0.7:
                    if artistAMtoDCScore is None:
                        artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                    else:
                        if retval > artistAMtoDCScore["Score"]:
                            artistAMtoDCScore = {"Name": artistDCName, "ID": artistDCID, "Score": retval}
                            
                            
        print("\tAM ID={0: <15}{1}".format(artistAMID, artistAMtoDCScore))
        if artistAMtoDCScore is not None:
            artistAMIDtoDCIDMap[artistAMID] = {"AM Name": artistAMName, "DC ID": artistAMtoDCScore["ID"], "DC Name": artistAMtoDCScore["Name"]}        
        
        
        print(artistName,allmusicIDs)
    continue
    for dbname, dirval in unknownVals.items():
        print("Partially Unknown Artist: {0: <40}{1: <15}{2}".format(artistName, dbname, dirval))
        artistAMNametoIDMap = {v["AM Name"]: k for k,v in artistAMIDtoDCIDMap.items()}

In [ ]:

        

        ## Print IDs
        printArtistIDs(artistName, discogsArtistIDX, allmusicArtistIDX)

        
        ## Get Database Entries
        discogsIDData  = getRowDataByColValue(discdfDC, "DiscArtist", artistName)
        allmusicIDData = getRowDataByColValue(discdfAM, "DiscArtist", artistName)

        
        ## Print Database ID Results
        printArtistIDDBResults(artistName, discogsIDData, allmusicIDData)
        
        
        
        ## Check for Match for all DBs
        discogsIDCnts  = getArtistIDDBCounts(discogsIDData)
        allmusicIDCnts = getArtistIDDBCounts(allmusicIDData)
        if discogsIDCnts == 1:
            musicMatchMap[artistName]["Status"].append("Discogs")
            musicMatchMap[artistName]["Matches"]["Discogs"]  = discogsIDData
            
        if allmusicIDCnts == 1:
            musicMatchMap[artistName]["Status"].append("AllMusic")
            musicMatchMap[artistName]["Matches"]["AllMusic"] = allmusicIDData

In [ ]:

        
        continue
        
        
        print(artistName,type(mdata))
        if not isinstance(mdata, DataFrame):
            matches = mulArts.getArtistNames(artistName)
            if not all(matches.values()):
                print("{0: <10}{1: <50}{2}".format("?", artistName, matches))
                continue
            else:
                print("{0: <10}{1: <50}{2}".format("MIX", artistName, matches))
                srcDir = dirval
                dstDir = setDir("/Volumes/Music/Multi", artistName)
                moveDir(srcDir, dstDir)                
        else:
            print('\t',mdata.shape)
            if mdata.shape[0] == 1:
                musicNameIDMap[artistName] = [mdata.index, mdata["Name"].values[0]]
            else:
                print("{0: <10}{1: <50}{2}".format("", artistName, ""))
                albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in discogMediaNames+myMediaNames]
                if len(albums) == 0:
                    print("\tNo Albums\n")
                    srcDir = dirval
                    dstDir = setDir("/Volumes/Music/Random", artistName)
                    moveDir(srcDir, dstDir)
                else:
                    multiMap[artistName] = {"DB": mdata, "Albums": albums}
                    continue
                    for album in albums:
                        print("\t",album)
                    print("")
                print("\tDiscogs Data:")
                print(mdata)
                print("\n\n")

# Manual Entries

In [ ]:
newManual = {}

In [ ]:
mdata = getMusicData("Name", "Daryl Hall & John Oates")
newManual[mdata["DiscArtist"].values[0]] = [mdata.index, mdata["Name"].values[0]]

In [ ]:
newManual

In [ ]:
if len(newManual) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in newManual.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in newManual.items()})
        except:
            raise ValueError("Not sure about {0}".format(newManual))

## Find MultiMap -> Match Entry

In [ ]:
ratVal = 0.6
if len(myMusicNameIDMap) > 0:
    ratVal = 0.3
artistIDMap = {}
for artistName,artistSlimData in multiMap.items():
    print("\n","="*50)
    print("ArtistName: {0}".format(artistName))
    print("   Albums: {0}".format(len(artistSlimData["Albums"])))

    myAlbumNames = []
    for album in artistSlimData["Albums"]:
        myAlbumName = album.split("/{0}/".format(artistName))[-1]
        #print("     {0: <15}{1: <10}{2}".format("", "", myAlbumName))
        myAlbumNames.append(myAlbumName)
        
    for idx,row in artistSlimData["DB"].iterrows():
        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]
        print("   Match: {0}  [{1}]".format(row["DiscArtist"], row["Name"]))
        if isinstance(artistAlbums, dict):
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    for myAlbumName in myAlbumNames:
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > 0.6:
                            print("     {0: <15}{1: <10}{2: <8}{3}".format(albumType, albumID, round(ratio,2), dbAlbumName))
                            artistIDMap[artistName] = [idx, row["Name"]]

## Found Multi Artist

In [ ]:
for artistName,artistData in artistIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
if len(artistIDMap) > 0:
    myMusicNameIDMap.update(artistIDMap)

## Set Single Artist

In [ ]:
for artistName,artistData in musicNameIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
print("Found {0} entries".format(len(musicNameIDMap)))
if len(musicNameIDMap) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in musicNameIDMap.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in musicNameIDMap.items()})
        except:
            raise ValueError("Not sure about {0}".format(musicNameIDMap))
print("Found {0} entries".format(len(musicNameIDMap)))
print("There are {0} total entries".format(len(myMusicNameIDMap)))

# Save Everything

In [ ]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)

In [ ]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

In [ ]:
myMusicNameIDMap = getFile("musicDiscogsMap.p")
print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

# Find Music and Match Albums

In [ ]:
#myMusicAlbumIDMap = getFile("musicDiscogsAlbumMap.p")
myMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

In [ ]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

## Johnny Cash

In [ ]:
#myMusicNameIDMap

# Show and Find Music <-> Discog Matches

In [ ]:
ratioCut = 0.65
keepSkips = False

In [ ]:
def showArtist(artistName, discogsArtist):
    print("\t{0: <30} ---> {1}".format(artistName, discogsArtist))
    
def showAlbum(myAlbumName):
    print("\t\tMy Album: {0}".format(myAlbumName))
    
def showMatch(myAlbumName, albumData):
    print("\t\t   Match: {0}".format(albumData))

musicAlbumIDMap = {}
discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
myMediaNames     = ['Random', 'Todo', 'Match', 'Title', 'Singles', 'Album', 'Unknown', 'Bootleg', 'Mix', 'Tribute']
ignoreDirs       = list(set(discogMediaNames+myMediaNames))
nSkips           = len(skipMusicAlbumIDMap)

numUnMatches = Counter()

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        try:
            discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
            discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        except:
            raise ValueError("Could not find Artist ID [{0}] and Artist Name [{1}]".format(idx, discogsArtistName))
        
        albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in ignoreDirs]        
        myAlbumNames = [album.split("/{0}/".format(artistName))[-1] for album in albums]
        
        numUnMatches[artistName] = len(myAlbumNames)
        
        for myAlbumName in myAlbumNames:
            myAlbumName = normalize('NFC', myAlbumName)
            if myMusicAlbumIDMap.get(artistName) is not None:
                if myMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    numUnMatches[artistName] -= 1
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue
                    
            if skipMusicAlbumIDMap.get(artistName) is not None:
                if skipMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    numUnMatches[artistName] -= 1
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue

            maxRatio  = None
            albumData = None
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 3)
                    if ratio > ratioCut:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio  = ratio
                            albumData = [albumID, dbAlbumName]
                            
            if albumData is not None:
                numUnMatches[artistName] -= 1
                if musicAlbumIDMap.get(artistName) is None:
                    musicAlbumIDMap[artistName] = {}
                    showArtist(artistName, discogsArtist)
                showAlbum(myAlbumName)
                musicAlbumIDMap[artistName][myAlbumName] = albumData
                showMatch(myAlbumName, albumData)
                        
            if keepSkips:
                if skipMusicAlbumIDMap.get(artistName) is None:
                    skipMusicAlbumIDMap[artistName] = {}
                #showAlbum(myAlbumName)
                skipMusicAlbumIDMap[artistName][myAlbumName] = [albumID, dbAlbumName]



## Check For Skips

In [ ]:
discogsArtist            = myMusicNameIDMap["John Fahey"]
idx                      = discogsArtist[0]
print("Artist ID -->",idx)
discogsArtistName        = discogsArtist[1]
discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]

for k,v in discogsArtistAlbums.items():
    for code,name in v.items():
        print("{0: <15}{1: <12}{2}".format(k,code,name))
        continue

In [ ]:
print("Previous {0} music <-> discogs albums maps".format(nSkips))
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)


In [ ]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

## Check For Match

In [ ]:
from ioUtils import saveFile
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

## Move To Matched

In [ ]:
from fsUtils import mkDir, isDir, moveDir
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        musicMap   = musicAlbumIDMap.get(artistName)
        if musicMap is None:
            continue
        for myAlbumName,albumVal in musicMap.items():
            matchedDir = setDir(dirval, "Match")
            mkDir(matchedDir)
            srcDir = setDir(dirval, myAlbumName)
            if not isDir(srcDir):
                print("{0} does not exist".format(srcDir))
                continue
            dstDir = setDir(matchedDir, discConv(myAlbumName))
            if isDir(dstDir):
                print("{0} already exists".format(dstDir))
                continue
                
            print("Moving {0}  --->  {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir)

# Move To Album Type

In [ ]:
moveData = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        matchedDir = setDir(dirval, "Match")
        aTypes     = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
        albums     = [getDirBasics(x)[-1] for x in findDirs(matchedDir)]
        albums     = [x for x in albums if x not in aTypes]
        if len(albums) == 0:
            continue
        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]
        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
        discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        
        
        for myAlbumName in albums:
            myAlbumName = normalize('NFC', myAlbumName)
            maxRatio    = None
            dbAlbumData = {}
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 2)
                    if ratio > 0.0:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio = ratio

                        if moveData.get(artistName) is None:
                            print('  ',artistName)
                            moveData[artistName] = {"Dir": matchedDir, "Albums": {}}
                        moveData[artistName]["Albums"][myAlbumName] = [albumID, albumType, convDiscogsAlbumName]
       
            if maxRatio is not None:
                print('\t{0: <50}'.format(myAlbumName),moveData[artistName]["Albums"][myAlbumName])

In [ ]:
saveFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", idata=moveData, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

In [ ]:
moveData = getFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

In [ ]:
atypes = {}
for artistName,artistMoveData in moveData.items():
    dirval = artistMoveData["Dir"]
    albums = artistMoveData["Albums"]
    for myAlbumName, dbAlbumData in albums.items():
        albumID, albumType, convDiscogsAlbumName = dbAlbumData[0], dbAlbumData[1], dbAlbumData[2]
        atypes[albumType] = True
        albumTypeDir = setDir(dirval, albumType)
        mkDir(albumTypeDir)
        srcDir  = setDir(dirval, myAlbumName)
        if not isDir(srcDir):
            continue

        dstName = " :: ".join([myAlbumName, "[{0}]".format(albumID)])
        dstDir  = setDir(albumTypeDir, dstName)
        if isDir(dstDir):
            print("{0} already exists.".format(dstDir))
            continue
        print("Moving {0}".format(srcDir))
        moveDir(srcDir, dstDir)

In [ ]:
list(atypes.keys())

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    print(artistName)
    for myAlbumName,v2 in v.items():
        print("\t",myAlbumName,'\t',v2)

In [ ]:
if True:
    saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)
    print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
skipMusicAlbumIDMap

In [ ]:
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

## Merge Maps (if needed)

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    for myAlbumName,v2 in v.items():
        if myMusicAlbumIDMap.get(artistName) is None:
            print("Adding {0}".format(artistName))
            myMusicAlbumIDMap[artistName] = {}
        if myMusicAlbumIDMap[artistName].get(myAlbumName) is None:
            print("Adding {0}/{1} ---> {2}".format(artistName, myAlbumName, v2))
            myMusicAlbumIDMap[artistName][myAlbumName] = v2

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsAlbumMap.p", idata=myMusicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

In [ ]:
saveFile(ifile="musicDiscogsAlbumMap.yaml", idata=myMusicAlbumIDMap, debug=True)

# Rename Albums

In [ ]:
musicAlbumIDMap = getFile(ifile="musicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
renames = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        if musicAlbumIDMap.get(artistName) is None:
            continue
        print("    {0}".format(artistName))
        for album, albumdata in musicAlbumIDMap[artistName].items():
            print("\t[{0}]   ====>   [{1}]".format(album,albumdata[1]))
            srcAlbumName = album
            dstAlbumName = discConv(albumdata[1])
            if srcAlbumName != dstAlbumName:
                if renames.get(dirval) is None:
                    renames[dirval] = {}
                renames[dirval][srcAlbumName] = dstAlbumName

In [ ]:
saveFile(ifile="musicRenames.yaml", idata=renames, debug=True)

In [ ]:
renameData = getFile(ifile="musicRenames.yaml", debug=True)

In [ ]:
from fsUtils import isDir
for dirval,artistdata in renameData.items():
    for src,dst in artistdata.items():
        srcDir = setDir(dirval, src)
        dstDir = setDir(dirval, dst)
        print(dstDir,isDir(dstDir))
        if srcDir != dstDir and not isDir(dstDir):
            print("Moving {0} -> {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir, debug=True)